# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_multiagent import Agent
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import torch

In [2]:
#place path to reacher unity env
path_to_unity_env = '~/Reacher_Linux_multi_agent/Reacher.x86_64'
env = UnityEnvironment(file_name=path_to_unity_env)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# What does the env look like

In [4]:
# reset the environment
env_itter = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_itter.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_itter.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [5]:
agent = Agent(state_size=33,
              action_size=4, random_seed=777)

In [ ]:
def ddpg(n_episodes=500, max_t=1000):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        #restart env 
        env_itter = env.reset(train_mode=True)[brain_name] #note training = True here
        #grab state of agents
        state =env_itter.vector_observations
        agent.reset()
        score = np.zeros(len(env_itter.agents)) 
        
        for t in range(max_t):
            #select an action for each agent based off learned weights
            
            action = agent.act(state,i_episode,add_noise=True)
            #send agents into env
            env_itter = env.step(action)[brain_name]  
            
            #pull out env results
            rewards = env_itter.rewards                      
            dones = env_itter.local_done
            next_states =env_itter.vector_observations
            
            #step agents forward in learning
            agent.step(state, action, rewards, next_states, dones)
            score += rewards      
            state = next_states 
            if np.any(dones):
                break 
    
        scores_deque.append(score)
        scores.append(score)

        print('\rEpisode {}\tAverage Score: {:.2f}\tMin Score: {:.2f}\tMax Score: {:.2f}\tEpisode score: {:.2f}'.format(i_episode, np.mean(scores_deque),np.min(scores_deque),np.max(scores_deque),np.mean(score)), end="")
        if i_episode % 5 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_longtrain.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_longtrain.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
        
    
    return scores


scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), np.mean(scores,axis=1))
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 5	Average Score: 1.38	Min Score: 0.00	Max Score: 4.60	Episode score: 2.55
Episode 10	Average Score: 2.27	Min Score: 0.00	Max Score: 8.78	Episode score: 4.37
Episode 15	Average Score: 3.79	Min Score: 0.00	Max Score: 12.40	Episode score: 7.40
Episode 20	Average Score: 5.56	Min Score: 0.00	Max Score: 16.47	Episode score: 13.08
Episode 25	Average Score: 7.64	Min Score: 0.00	Max Score: 22.06	Episode score: 17.72
Episode 30	Average Score: 9.54	Min Score: 0.00	Max Score: 34.32	Episode score: 18.63
Episode 35	Average Score: 11.08	Min Score: 0.00	Max Score: 34.32	Episode score: 20.40
Episode 40	Average Score: 12.25	Min Score: 0.00	Max Score: 34.32	Episode score: 20.47
Episode 45	Average Score: 13.15	Min Score: 0.00	Max Score: 35.20	Episode score: 20.34
Episode 50	Average Score: 13.94	Min Score: 0.00	Max Score: 39.22	Episode score: 21.86
Episode 55	Average Score: 14.72	Min Score: 0.00	Max Score: 39.22	Episode score: 22.70
Episode 60	Average Score: 15.37	Min Score: 0.00	Max Score: 39.22	E